<h1><center>ResNet-50识别岩石样本</center></h1>

In [1]:
# 导包
#import tensorflow.compat.v1 as tf
my_seed = 520
# 导包
import os
os.environ['PYTHONHASHSEED'] = str(my_seed)

import numpy as np
import random 
import tensorflow as tf
import imgaug as ia

# 设置随机种子，保证可复现, 随机种子的设置要在任何一次使用到随机数前设置，因此通常在刚导入完所有包后立即设置。提示：后续的解决样本不平衡过程。
np.random.seed(my_seed)
random.seed(my_seed)
ia.seed(my_seed)
# tf.random.set_seed(my_seed)

import keras as K
from keras import backend
import pandas as pd
from keras import models,layers 
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import preprocess_input,decode_predictions
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from matplotlib import pyplot as plt 
from PIL import Image
from imgaug import augmenters as iaa
# from numba import cuda
import cv2
%matplotlib inline 

D:\Application\Anaconda\envs\keras\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
Using TensorFlow backend.


## 模型搭建

In [2]:
# 引入ResNet网络

num_classes = 7  # rock为7类
def ResNet50Model():
    model = K.applications.ResNet50(include_top=False, weights="imagenet", input_shape =(224, 224, 3))
    
    new_output = K.layers.GlobalAveragePooling2D()(model.output)
    new_output = K.layers.Dense(num_classes, activation = 'softmax')(new_output)
    
    model = K.engine.training.Model(model.inputs, new_output)
    
    return model

model = ResNet50Model()
print("num_layers = ", len(model.layers))

D:\Application\Anaconda\envs\keras\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


num_layers =  177


In [3]:
# model.summary()

In [4]:
# 冻结前168层，但不冻结其间的Batch Normalization
num_feerezed_layer = 168

for layer in model.layers:
    layer.trainable = True

    if isinstance(layer, K.layers.BatchNormalization):
        layer.momentum = 0.9    # 认为动量为0.9时能更好的适应新的数据集

for layer in model.layers[:num_feerezed_layer]:

    if not isinstance(layer, K.layers.BatchNormalization):
        layer.trainable = True

# 检查可训练层冻结是否正确
for layer in model.layers:
    print(layer.name, " ", "trainable = ", layer.trainable)

input_1   trainable =  True
conv1_pad   trainable =  True
conv1   trainable =  True
bn_conv1   trainable =  True
activation_1   trainable =  True
pool1_pad   trainable =  True
max_pooling2d_1   trainable =  True
res2a_branch2a   trainable =  True
bn2a_branch2a   trainable =  True
activation_2   trainable =  True
res2a_branch2b   trainable =  True
bn2a_branch2b   trainable =  True
activation_3   trainable =  True
res2a_branch2c   trainable =  True
res2a_branch1   trainable =  True
bn2a_branch2c   trainable =  True
bn2a_branch1   trainable =  True
add_1   trainable =  True
activation_4   trainable =  True
res2b_branch2a   trainable =  True
bn2b_branch2a   trainable =  True
activation_5   trainable =  True
res2b_branch2b   trainable =  True
bn2b_branch2b   trainable =  True
activation_6   trainable =  True
res2b_branch2c   trainable =  True
bn2b_branch2c   trainable =  True
add_2   trainable =  True
activation_7   trainable =  True
res2c_branch2a   trainable =  True
bn2c_branch2a   traina

## 模型训练

In [5]:
# 载入训练数据
rock_path = './train_1'
label_path = './label_new.csv'
label = pd.read_csv(label_path)

In [6]:
#计算最多的样本数
num_cls = []
for cls in set(label['类别标签']):
    num_cls.append([cls,len(label[label['类别标签']==cls])])
max_num_cls = max([i[1] for i in num_cls])
print(num_cls)
print('样本数最多的是',max_num_cls)

[[0, 85], [1, 18], [2, 30], [3, 75], [4, 46], [5, 40], [6, 21]]
样本数最多的是 85


In [7]:
# 最简单的解决样本不平衡方法一：复制小比例样本
for cls in range(len(num_cls)):
    label_cls = label[label['类别标签']==cls]
    residual_num = max_num_cls-len(label_cls) # 计算要补充的数量
    residual_index = np.random.choice(range(len(label_cls)),residual_num)
    label_residual = label_cls.iloc[residual_index]
    label = pd.concat([label,label_residual],ignore_index = True)
random_index = np.arange(len(label))
random.shuffle(random_index)
label = label.iloc[random_index]
label.index = range(len(label))
label

,样本编号,样本类别,类别标签
0,257,灰黑色泥岩,2
1,22,灰色泥质粉砂岩,4
2,22,灰色泥质粉砂岩,4
3,83,黑色煤,6
4,21,灰黑色泥岩,2
...,...,...,...
590,153,深灰色粉砂质泥岩,5
591,179,灰黑色泥岩,2
592,179,灰黑色泥岩,2
593,67,深灰色泥岩,3


In [8]:
# 23 ，344 ， 109

In [9]:
img_pathes = ['{}-1.bmp'.format(i) if i<=320 else '{}-1.jpg'.format(i) for i in label['样本编号']  ]
img_pathes = [os.path.join(rock_path,i) for i in img_pathes]# 补全路径,全部图片的路径
label = label['类别标签']
label = list(label)

In [10]:
# 分割训练集和测试集
val_ratio = 0.2
x_train, x_val, y_train, y_val = train_test_split(img_pathes, label, test_size=val_ratio, 
                                                  random_state=42, shuffle=True, stratify=label)

In [11]:
# 归一化，并且把输入的图片从RGB模式改为BGR模式
def imgsub(img):
    img_res = np.zeros(img.shape)          # R G B
    img_res[:,:,0] = img[:,:,2]            # B = B - B
    img_res[:,:,1] = img[:,:,1]            # G = G - G
    img_res[:,:,2] = img[:,:,0]            # R = R - R
    return img_res  # B G R

In [12]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # 0.5的概率水平翻转
    iaa.Flipud(0.5),
    iaa.Crop(percent=(0, 0.2)), # random crops
    iaa.ContrastNormalization((0.75, 1.25)),
    iaa.Multiply((0.75, 1.5)),
    # 对每张图片进行仿射变换，包括缩放、平移、旋转、修剪等
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-180, 180),
        shear=(-8, 8)
    )
], random_order=True) # 随机应用以上的图片增强方法

# 定义生成器，用于训练时batch的生成（小批量生成可以避免内存不够）
def Generator(img_set,labels,batch_size,return_names = False):
    while True:
        img_batch = []
        label_batch = []
        path_batch = []
        for i in range(len(img_set)):
            img_path = img_set[i]
            img = Image.open(img_path)
            img = img.convert('RGB')
            img = img.resize((224,224))
            img = np.asarray(img)
            img_batch.append(img)

            label = labels[i]
            label_batch.append(label)
            path_batch.append(img_path)
            if (i+1)%batch_size==0 or (i+1) == len(img_set):# 凑够batch_size张图片的矩阵
                img_batch = np.array(img_batch) # 拼成一个大矩阵，用于训练
                img_batch = seq.augment_images(img_batch.astype(np.uint8)).astype(np.float64)
                for i in range(img_batch.shape[0]):
                    img_batch[i] = imgsub(img_batch[i])
                label_batch = np.array(label_batch)
                label_batch = K.utils.to_categorical(label_batch, num_classes=7)
                if not return_names:
                    yield img_batch,label_batch#,img_batch_before_aug
                else:
                    yield img_batch,label_batch,path_batch
                img_batch = []
                label_batch = []
                path_batch = []

D:\Application\Anaconda\envs\keras\lib\site-packages\imgaug\imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)


### 找最优学习率

In [1]:
# if not os.path.exists('Best_rock_Model(ResNet).h5'):
if True:
    model = ResNet50Model()

    lr_begin = 1e-12
    lr_end = 0.1

    batch_size = 5
    train_g = Generator(x_train, y_train, batch_size)
    batch_num = len(x_train)//batch_size+1 # 一个epoch所含有的batch数量

    lr = lr_begin
    lr_multi = (lr_end/lr_begin)**(1/batch_num)
    lr_all = []
    loss_all = []
    acc_all = []

    optim = K.optimizers.Adamax(lr=lr)
    model.compile(loss='categorical_crossentropy', optimizer=optim,
                      metrics=['accuracy'])
    cnt = 0
    for batch_data, batch_y in train_g:
        cnt+=1
        backend.set_value(model.optimizer.lr, lr)
        loss,acc = model.train_on_batch(batch_data,batch_y)
        lr_all.append(lr)
        loss_all.append(loss)
        acc_all.append(acc)
        lr = lr*lr_multi  

        if cnt == batch_num:
            break

    lr_all = np.array(lr_all)
    lr_best = lr_all[np.argmin(loss_all)]
    print('最优学习率为:',lr_best)
    plt.plot(np.log10(lr_all),loss_all)
    plt.xticks(np.log10([ 1e-10, 1e-8, 1e-6, 1e-4, 1e-3, 1e-2,1e-1]), [ 1e-10, 1e-8, 1e-6, 1e-4, 1e-3, 1e-2,1e-1])
    plt.xlim(np.log10(1e-10),np.log10(1e-2))
    plt.ylim(0,3)
    plt.ylabel('loss')
    plt.xlabel('learning rate')
    plt.savefig('find_lr_520.png',dpi  =100 )
    plt.show()

NameError: name 'ResNet50Model' is not defined

In [16]:
lr_best = 1e-3 # 还是肉眼手动根据绘图结果指定lr_best比较好一些，肉眼可以综合考虑学习率在训练时的稳定性

if os.path.exists('Best_rock_Model(ResNet).h5'):
    model_Best = 'Best_rock_Model(ResNet).h5'
    model = load_model(model_Best)
    optim = K.optimizers.Adam(lr=1e-4)
else:
    model = ResNet50Model()
    optim = K.optimizers.Adam(lr=1e-4)#lr_best)

# 定义学习率调整方案
def scheduler(epoch):
    lr = optim.lr.numpy()
#     if epoch>=10:
#         lr = 1e-4
    if epoch>=3:
        lr = 1e-6
    print(epoch, lr)
    return lr 
reduce_lr = K.callbacks.LearningRateScheduler(scheduler)

# 断点续训
checkpoint = ModelCheckpoint('Best_rock_Model(ResNet).h5', verbose=1, monitor='val_accuracy', 
                             save_best_only=True, mode='max')

# 指定模型的损失函数，优化器，衡量指标

model.compile(loss='categorical_crossentropy', optimizer=optim,
              metrics=['accuracy'])

# 模型训练
batch_size = 5
history = model.fit(Generator(x_train, y_train, batch_size),
              steps_per_epoch=len(x_train)//batch_size,
              epochs=50,
              validation_data=Generator(x_val, y_val, batch_size),
              validation_steps=len(x_val) // batch_size,
#               class_weight = 'auto',
              callbacks=[checkpoint,reduce_lr])

D:\Application\Anaconda\envs\keras\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Epoch 1/50
0 1e-04
95/95 [==============================] - 30s 318ms/step - loss: 1.5476 - accuracy: 0.4147 - val_loss: 1.7566 - val_accuracy: 0.4348

Epoch 00001: val_accuracy improved from -inf to 0.43478, saving model to Best_rock_Model(ResNet).h5
Epoch 2/50
1 1e-04
95/95 [==============================] - 20s 209ms/step - loss: 1.2507 - accuracy: 0.5350 - val_loss: 1.2077 - val_accuracy: 0.5702

Epoch 00002: val_accuracy improved from 0.43478 to 0.57018, saving model to Best_rock_Model(ResNet).h5
Epoch 3/50
2 1e-04
95/95 [==============================] - 20s 207ms/step - loss: 1.1371 - accuracy: 0.5796 - val_loss: 3.0130 - val_accuracy: 0.6053

Epoch 00003: val_accuracy improved from 0.57018 to 0.60526, saving model to Best_rock_Model(ResNet).h5
Epoch 4/50
3 1e-06
95/95 [==============================] - 20s 207ms/step - loss: 1.0529 - accuracy: 0.6093 - val_loss: 0.9955 - val_accuracy: 0.6140

Epoch 00004: val_accuracy improved from 0.60526 to 0.61404, saving model to Best_rock_

95/95 [==============================] - 20s 206ms/step - loss: 0.8115 - accuracy: 0.7219 - val_loss: 0.2592 - val_accuracy: 0.6579

Epoch 00039: val_accuracy did not improve from 0.66667
Epoch 40/50
39 1e-06
95/95 [==============================] - 20s 206ms/step - loss: 0.7928 - accuracy: 0.7197 - val_loss: 0.6019 - val_accuracy: 0.5614

Epoch 00040: val_accuracy did not improve from 0.66667
Epoch 41/50
40 1e-06
95/95 [==============================] - 20s 207ms/step - loss: 0.7989 - accuracy: 0.7070 - val_loss: 0.7618 - val_accuracy: 0.6316

Epoch 00041: val_accuracy did not improve from 0.66667
Epoch 42/50
41 1e-06
95/95 [==============================] - 20s 207ms/step - loss: 0.7571 - accuracy: 0.7091 - val_loss: 1.7426 - val_accuracy: 0.6491

Epoch 00042: val_accuracy did not improve from 0.66667
Epoch 43/50
42 1e-06
95/95 [==============================] - 20s 206ms/step - loss: 0.7579 - accuracy: 0.7219 - val_loss: 1.3351 - val_accuracy: 0.6316

Epoch 00043: val_accuracy did n

In [ ]:
# 绘制训练精度和验证精度
history_dict = history.history
acc = history_dict['accuracy']
val_accuracy = history_dict['val_accuracy']

epochs = range(1, len(acc)+1)


plt.plot(epochs, acc, label = 'Training accuracy')
plt.plot(epochs, val_accuracy, label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('seed_520_acc.png',dpi=200) # 保存图片
plt.show()

In [ ]:
# 绘制训练损失和验证损失
history_dict = history.history
acc = history_dict['loss']
val_accuracy = history_dict['val_loss']

epochs = range(1, len(acc)+1)


plt.plot(epochs, acc, label = 'Training loss')
plt.plot(epochs, val_accuracy, label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('seed_520_loss.png',dpi = 200)
plt.show()

## 模型预测

In [ ]:
# 载入图片,进行预测
r = './rock/1-1.bmp'
img = Image.open(r)
img = img.resize((224,224))
img = np.array(img)
img = img[:,:,:3]
img = imgsub(img)
img = np.expand_dims(img,axis = 0 )


if os.path.exists('Best_rock_Model(ResNet).h5'):
    model_Best = 'Best_rock_Model(ResNet).h5'
    model = load_model(model_Best)

res = model.predict(img)

# 预测结果
print('Results are',res)

## 预测结果可视化

In [ ]:
# plt绘图中显示中文和符号
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False 

df_label = pd.read_csv('label_new.csv')

# fig = plt.figure(figsize = (15,4*315))
for i in range(len(x_val)):
    img_show_name = x_val[i]
    img_show_num = os.path.split(img_show_name)[-1].split('-')[0]
    img_show_num = int(img_show_num)
    
    # 利用样本编号（文件名,x_val）与样本类别的对应关系
    label_show_chinese = df_label['样本类别'][df_label['样本编号']==img_show_num]
    label_show_chinese = label_show_chinese.iloc[0]
    
    img = Image.open(img_show_name).convert('RGB')
    img = img.resize((224,224))
    img = np.array(img)
    img_plot = img.copy()
    
    img = imgsub(img)
    img = np.expand_dims(img,axis = 0 )
    predict = model.predict(img)
    predict_num = np.argmax(predict)
    predict_chinese = df_label['样本类别'][df_label['类别标签']==predict_num].iloc[0]# 得到预测结果的中文
    
    #ax = fig.add_subplot(315,5,i+1)
    if i%5==0:
        plt.figure(figsize = (15,3))
    plt.subplot(1,5,i%5+1)
    img_plot = Image.fromarray(img_plot).resize((800,600))
    img_plot = np.array(img_plot)
    plt.imshow(img_plot)
    plt.title(f'predict：{predict_chinese}\nground truth：{label_show_chinese}')
    

# 生成器数据检查

In [ ]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # 0.5的概率水平翻转
    iaa.Crop(percent=(0, 0.2)), # random crops
    #sigma在0~0.5间随机高斯模糊，且每张图纸生效的概率是0.5
#     iaa.Sometimes(0.5,
#         iaa.GaussianBlur(sigma=(0, 0.5))
#     ),
    # 增大或减小每张图像的对比度
    iaa.ContrastNormalization((0.9, 1.2)),
    iaa.Multiply((1, 1.2), per_channel=0.2),
    # 对每张图片进行仿射变换，包括缩放、平移、旋转、修剪等
    iaa.Affine(
#         scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) # 随机应用以上的图片增强方法

# 定义生成器，用于训练时batch的生成（小批量生成可以避免内存不够）
def Generator(img_set,labels,batch_size):
    while True:
        img_batch = []
        label_batch = []
        for i in range(len(img_set)):
            img_path = img_set[i]
            img = Image.open(img_path)
            img = img.convert('RGB')
            img = img.resize((224,224))
            img = np.asarray(img)
#             img = imgsub(img)
            img_batch.append(img)

            label = labels[i]
            label_batch.append(label)
            if (i+1)%batch_size==0 or (i+1) == len(img_set):# 凑够batch_size张图片的矩阵
                img_batch_before_aug = np.array(img_batch) # 拼成一个大矩阵，用于训练
                img_batch = seq.augment_images(img_batch_before_aug.astype(np.uint8))
#                 for i in range(img_batch.shape[0]):
#                     img_batch[i] = imgsub(img_batch[i])
                label_batch = np.array(label_batch)
                label_batch = K.utils.to_categorical(label_batch, num_classes=7)
                yield img_batch,label_batch,img_batch_before_aug

                img_batch = []
                label_batch = []

In [ ]:
train_gen = Generator(x_train, y_train, batch_size)
cnt = 0 
for img_batch,label_batch,img_before in train_gen:
    plt.subplot(121)
    plt.imshow(img_before[0])
    plt.subplot(122)
    plt.imshow(img_batch[0])
    plt.show()
    cnt+=1
    if cnt==20:
        break

# 增强数据集本地保存

In [ ]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # 0.5的概率水平翻转
    iaa.Flipud(0.5),
    iaa.Crop(percent=(0, 0.2)), # random crops
    iaa.ContrastNormalization((0.75, 1.5)),
    iaa.Multiply((0.75, 2)),
    # 对每张图片进行仿射变换，包括缩放、平移、旋转、修剪等
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-180, 180),
        shear=(-8, 8)
    )
], random_order=True) # 随机应用以上的图片增强方法

# 定义生成器，用于训练时batch的生成（小批量生成可以避免内存不够）
def Generator(img_set,labels,batch_size,return_names = False):
#     while True:
    img_batch = []
    label_batch = []
    path_batch = []
    for i in range(len(img_set)):
        img_path = img_set[i]
        img = Image.open(img_path)
        img = img.convert('RGB')
        img = img.resize((224,224))
        img = np.asarray(img)
        img_batch.append(img)

        label = labels[i]
        label_batch.append(label)
        path_batch.append(img_path)
        if (i+1)%batch_size==0 or (i+1) == len(img_set):# 凑够batch_size张图片的矩阵
            img_batch = np.array(img_batch) # 拼成一个大矩阵，用于训练
            img_batch = seq.augment_images(img_batch.astype(np.uint8)).astype(np.float64)
#                 for i in range(img_batch.shape[0]):
#                     img_batch[i] = imgsub(img_batch[i])
            label_batch = np.array(label_batch)
            label_batch = K.utils.to_categorical(label_batch, num_classes=7)
            if not return_names:
                yield img_batch,label_batch#,img_batch_before_aug
            else:
                yield img_batch,label_batch,path_batch
            img_batch = []
            label_batch = []
            path_batch = []

In [ ]:
if not os.path.exists('test_aug'):
    os.mkdir('test_aug')
if not os.path.exists('train_aug'):
    os.mkdir('train_aug')
    
train_gen = Generator(x_train,y_train,5,return_names = True)
cnt = 0
for img_batch,label_batch,path_batch in train_gen:
    for i in range(img_batch.shape[0]):
        label = np.argmax(label_batch[i])
        img_path = os.path.join('train_aug',str(int(label)),os.path.split(path_batch[i])[-1].split('.')[0]+'_'+str(cnt)+'.png')
        if not os.path.exists(os.path.split(img_path)[0]):
            os.makedirs(os.path.split(img_path)[0])            
        Image.fromarray(img_batch[i].astype(np.uint8)).save(img_path)
        print(img_path,'is saved.')
        cnt += 1
#         if cnt==len(x_train):
#             break
        
test_gen = Generator(x_val,y_val,5,return_names = True)
cnt = 0
for img_batch,label_batch,path_batch in test_gen:
    for i in range(img_batch.shape[0]):
        label = np.argmax(label_batch[i])
        img_path = os.path.join('test_aug',str(int(label)),os.path.split(path_batch[i])[-1].split('.')[0]+'_'+str(cnt)+'.png')
        if not os.path.exists(os.path.split(img_path)[0]):
            os.makedirs(os.path.split(img_path)[0])
        Image.fromarray(img_batch[i].astype(np.uint8)).save(img_path)
        print(img_path,'is saved.')
        cnt += 1
#         if cnt==len(x_val):
#             break